# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( r'F:\Data Science\data_wrangling_xml\data\mondial_database_less.xml' )

In [4]:
for element in document_tree.iterfind('country'):
    print(element.attrib['car_code'])
        

AL
GR
MK
SRB
MNE
KOS
AND


In [55]:
arr

[['Tirana', '1987', '192000'],
 ['Tirana', '1990', '244153'],
 ['Tirana', '2011', '418495'],
 ['Shkodër', '1987', '62000'],
 ['Shkodër', '2011', '77075'],
 ['Durrës', '1987', '60000'],
 ['Durrës', '2011', '113249'],
 ['Vlorë', '1987', '56000'],
 ['Vlorë', '2011', '79513'],
 ['Elbasan', '1987', '53000'],
 ['Elbasan', '2011', '78703'],
 ['Korçë', '1987', '52000'],
 ['Korçë', '2011', '51152'],
 ['Kavala', '1981', '56705'],
 ['Kavala', '1991', '60187'],
 ['Kavala', '2001', '63774'],
 ['Kavala', '2011', '58790'],
 ['Athina', '1981', '885737'],
 ['Athina', '1991', '816556'],
 ['Athina', '2001', '789166'],
 ['Athina', '2011', '664046'],
 ['Peiraias', '1981', '196389'],
 ['Peiraias', '1991', '187399'],
 ['Peiraias', '2001', '181933'],
 ['Peiraias', '2011', '163688'],
 ['Peristeri', '1991', '141971'],
 ['Peristeri', '2001', '146743'],
 ['Peristeri', '2011', '139981'],
 ['Acharnes', '1991', '65035'],
 ['Acharnes', '2001', '82555'],
 ['Acharnes', '2011', '106943'],
 ['Patra', '1981', '142163'],
 

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [25]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [4]:
document = ET.parse( r'F:\Data Science\data_wrangling_xml\data\mondial_database.xml' )

# 10 countries with the lowest infant mortality rates

## Creating an array with country name and infant mortality

In [5]:
import pandas as pd
import numpy as np

from xml.etree.ElementTree import QName
# print names of all countries and their cities
a=[]
for element in document.iterfind('country'):
    if element.find('infant_mortality') is None:
        a.append([element.find('name').text,''])
        continue
    a.append([element.find('name').text,element.find('infant_mortality').text])


## Changing array into Dataframe

In [48]:
a=pd.DataFrame(a)
a.columns=['name','val']

## changing datatype of val columns to numeric

In [49]:
a.val=a.val.apply(pd.to_numeric)

## Removing NaN from val

In [50]:
a=a[~np.isnan(a.val)]

## Getting last 10 record 

In [51]:
a.sort_values('val').head(10)

,name,val
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


## 10 cities with the largest population

## Create an array with city name, year and population in that year

In [56]:
arr = []
for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        for subsubelement in subelement.getiterator('population'):
            arr.append([subelement.find('name').text,subsubelement.attrib['year'],subsubelement.text])

## Create a Dataframe from array

In [64]:
pop_df=pd.DataFrame(arr)
pop_df.columns=['City','Year','Population']

##Change the datatype of Population column 

In [66]:
pop_df.Population=pop_df.Population.apply(pd.to_numeric)

## Get the tp 10 populated cities 

In [68]:
pop_df.sort_values('Population',ascending=False).head(10)

,City,Year,Population
3750,Shanghai,2010,22315474
3749,Shanghai,2000,15758892
2607,Istanbul,2012,13710512
4398,Delhi,2001,12877470
4303,Mumbai,2011,12442373
1546,Moskva,2013,11979529
4302,Mumbai,2001,11914398
3746,Beijing,2010,11716620
1545,Moskva,2010,11612885
8208,São Paulo,2010,11152344


# name and country of a) longest river, b) largest lake and c) airport at highest elevation

## Getting the river name, length and country name from the xml 

In [27]:
river_len=[]
for element in document.iterfind('river'):
    if element.find('length') is None:
        continue
    river_len.append([element.find('name').text,element.find('length').text,element.attrib['country']])

In [28]:
river_len=pd.DataFrame(river_len)

## Getting the longest liver 

In [29]:
river_len.columns=['name','length','country']
river_len.length=river_len.length.apply(pd.to_numeric)
river_len.sort_values('length',ascending=False).head(1)

,name,length,country
174,Amazonas,6448.0,CO BR PE


In [45]:
lake_len=[]
for element in document.iterfind('lake'):
    lake_len.append([element.find('name').text,element.find('latitude').text,element.attrib['country']])

In [48]:
lake_len=pd.DataFrame(lake_len)
lake_len.columns=['name','length','country']

In [49]:
lake_len.length=lake_len.length.apply(pd.to_numeric)
lake_len.sort_values('length',ascending=False).head(1)

,name,length,country
44,Ozero Taimyr,74.5,R


In [6]:
airport=[]
for element in document.iterfind('airport'):
    airport.append([element.find('name').text,element.find('elevation').text,element.attrib['country']])

In [7]:
airport=pd.DataFrame(airport)

In [8]:
airport.columns=['name','elevation','country']

In [18]:
airport.elevation=airport.elevation.astype(float)

In [20]:
airport.sort_values('elevation', ascending=False).head()

,name,elevation,country
80,El Alto Intl,4063.0,BOL
219,Lhasa-Gonggar,4005.0,CN
241,Yushu Batang,3963.0,CN
813,Juliaca,3827.0,PE
815,Teniente Alejandro Velasco Astete Intl,3311.0,PE


In [50]:
con_name=[]
for element in document.iterfind('country'):
    con_name.append([element.attrib['car_code'],element.find('name').text])

In [51]:
con_name=pd.DataFrame(con_name)

In [52]:
con_name.columns=['code','name']


In [53]:
con_name[con_name.code=='R']

,code,name
23,R,Russia
